In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from postgres import connection
import psycopg2
from datetime import datetime

In [2]:

def download_data(connection: psycopg2.extensions.connection, start_date = "2009-12-01", end_date = "2022-12-30") -> pd.DataFrame:
    cursor = connection.cursor()
    cursor.execute("SELECT ticker FROM equities")
    tickers = cursor.fetchall()
    stock_data = {}

    for stock_tuple in tickers:
        stock = stock_tuple[0]
        if stock == "^STI":
            continue
        table_name = f"stock_{stock[:3]}"  # Remove the .SI suffix
        query = (
            f"SELECT * FROM {table_name} WHERE Date >= %s AND Date <= %s ORDER BY Date ASC"
        )
        cursor.execute(query, (start_date, end_date))
        data = cursor.fetchall()
        if data:
            dates, open_price, high, low, close, adj_close, volume = zip(*data)
            
            if datetime.strptime(start_date, "%Y-%m-%d") != datetime.strptime(dates[0], "%Y-%m-%d")\
                or datetime.strptime(end_date, "%Y-%m-%d") != datetime.strptime(dates[-1], "%Y-%m-%d"):
                print(f"Insufficient data for {stock}. Data not retrieved. {dates[0]} to {dates[-1]}")
                continue
            
            
            df = pd.DataFrame({
                'Date': pd.to_datetime(dates),
                'Open': open_price,
                'High': high,
                'Low': low,
                'Close': close,
                'Adj_Close': adj_close,
                'Volume': volume
            })
            df.replace(0, np.nan, inplace=True)
            stock_data[stock] = df

    cursor.close()

    return stock_data

In [3]:
data = download_data(connection)
print(f"{len(data)} stocks downloaded")
for key, val in data.items():
    null_count = val.isnull().sum()
    for column, count in null_count.items():
        if count > 0 and column != 'Volume':
            print(f"Ticker: {key}")
            print(f"Column: {column}, Null Count: {count}")

Insufficient data for 43F.SI. Data not retrieved. 2016-04-18 to 2022-12-30
Insufficient data for 40V.SI. Data not retrieved. 2010-07-05 to 2022-12-30
Insufficient data for 1D4.SI. Data not retrieved. 2017-04-27 to 2022-12-30
Insufficient data for CLN.SI. Data not retrieved. 2017-09-28 to 2022-12-30
Insufficient data for XZL.SI. Data not retrieved. 2019-05-09 to 2022-12-30
Insufficient data for 505.SI. Data not retrieved. 2009-12-02 to 2022-12-30
Insufficient data for S7OU.SI. Data not retrieved. 2013-05-29 to 2022-12-30
Insufficient data for 1F3.SI. Data not retrieved. 2017-07-31 to 2022-12-30
Insufficient data for 5UF.SI. Data not retrieved. 2012-06-22 to 2022-12-30
Insufficient data for 8AZ.SI. Data not retrieved. 2021-03-12 to 2022-12-30
Insufficient data for P8Z.SI. Data not retrieved. 2012-04-12 to 2022-12-30
Insufficient data for 9CI.SI. Data not retrieved. 2021-09-20 to 2022-12-30
Insufficient data for 573.SI. Data not retrieved. 2009-12-02 to 2022-12-30
Insufficient data for P9

In [4]:
connection.close()